In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import brentq
from pathlib import Path
from IPython.display import display, Markdown
try:
    import yfinance as yf
    YFINANCE_AVAILABLE = True
except ImportError:
    YFINANCE_AVAILABLE = False

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8), 'figure.dpi': 150})
np.set_printoptions(suppress=True, linewidth=120, precision=4)
warnings.filterwarnings('ignore')

# --- Utility Functions ---
def note(msg, **kwargs): display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title): print(f"\n{80*'='}\n| {title.upper()} |\n{80*'='}")

note("Environment initialized for Option Pricing.")

# Chapter 9.4: Option Pricing: From Binomial Trees to Black-Scholes

---

### Table of Contents

1.  [**Introduction to Options and Payoffs**](#intro)
    - [Call and Put Option Payoff Diagrams](#payoffs)
2.  [**The No-Arbitrage Principle**](#no-arbitrage)
3.  [**Model 1: The Binomial Asset Pricing Model**](#binomial)
    - [Constructing a Binomial Tree](#binomial-tree)
    - [The Replicating Portfolio and Risk-Neutral Pricing](#replication)
4.  [**Model 2: The Black-Scholes-Merton (BSM) Model**](#bsm)
    - [Deriving the BSM Partial Differential Equation](#bsm-pde)
    - [The BSM Formula for European Options](#bsm-formula)
5.  [**Risk Management: The Greeks**](#greeks)
    - [Visualizing the Greeks](#greeks-viz)
6.  [**Model 3: Monte Carlo Simulation for Option Pricing**](#monte-carlo)
    - [Pricing Exotic Options: Asian and Barrier Options](#exotics)
7.  [**Real-World Application: The Volatility Smile**](#vol-smile)
    - [Case Study: Calculating Implied Volatility for AAPL Options](#case-study)
8.  [**Beyond Black-Scholes: Handling the Smile**](#beyond-bsm)
9.  [**Exercises**](#exercises)

<a id='intro'></a>
## 1. Introduction to Options and Payoffs

A financial **option** is a derivative contract that gives the buyer the **right, but not the obligation**, to buy (a **call** option) or sell (a **put** option) an underlying asset at a specified **strike price** ($K$) on or before a specified **expiration date** ($T$). The price paid for this right is called the **premium**.

- **Call Option:** Gives the holder the right to *buy* the underlying asset. A call option is profitable if the asset price $S_T$ at expiration is above the strike price $K$.
- **Put Option:** Gives the holder the right to *sell* the underlying asset. A put option is profitable if the asset price $S_T$ at expiration is below the strike price $K$.

The value of an option at expiration is its **intrinsic value** or **payoff**.

<a id='payoffs'></a>
### Call and Put Option Payoff Diagrams

The payoff functions at expiration ($T$) for a call and a put option are:
$$ \text{Call Payoff} = \max(S_T - K, 0) $$ 
$$ \text{Put Payoff} = \max(K - S_T, 0) $$ 
We can visualize these payoffs:

![Call and Put Payoffs](images/finance/options/call_put_payoffs.png)
*Figure 1: Payoff diagrams for a long call and a long put option, showing the profit and loss zones relative to the strike price.*

<a id='no-arbitrage'></a>
## 2. The No-Arbitrage Principle

How do we determine a fair price for an option *before* expiration? The unifying principle is **no-arbitrage**: in an efficient market, there should be no strategy that generates a positive, risk-free profit with zero initial investment. This implies that the price of any derivative must equal the cost of a **replicating portfolio**—a portfolio of the underlying asset and a risk-free bond that perfectly mimics the derivative's payoffs in all future states of the world. If the derivative were cheaper than the replicating portfolio, an arbitrageur could buy the derivative, sell the replicating portfolio, and lock in a risk-free profit.

This leads to the powerful concept of **risk-neutral valuation**. Since we can perfectly hedge the risk, investors' risk preferences do not affect the price. We can therefore value the option in a fictional "risk-neutral world" where all assets are expected to grow at the risk-free rate. This simplifies the problem immensely and forms the basis for the models we will explore.

<a id='binomial'></a>
## 3. Model 1: The Binomial Asset Pricing Model

<a id='binomial-tree'></a>
### Constructing a Binomial Tree
The Binomial Model, developed by Cox, Ross, and Rubinstein (1979), discretizes time into a series of steps. In each step, the underlying asset price $S$ is assumed to either move up by a factor $u > 1$ or down by a factor $d < 1$. We can form a **binomial tree** representing all possible price paths.

To ensure the tree is arbitrage-free and matches the volatility of the underlying asset, the parameters are defined as:
- **Up-factor ($u$):** $ u = e^{\sigma \sqrt{\Delta t}} $
- **Down-factor ($d$):** $ d = e^{-\sigma \sqrt{\Delta t}} = 1/u $
- **Risk-Neutral Probability ($p$):** This is the probability of an up-move in a world where the asset grows at the risk-free rate. It is *not* the real-world probability.
$$ p = \frac{e^{r \Delta t} - d}{u - d} $$

<a id='replication'></a>
### The Replicating Portfolio and Risk-Neutral Pricing

The core insight is that at any node in the tree, we can form a portfolio of $\Delta$ shares of the stock and $B$ dollars in a risk-free bond that exactly replicates the payoff of the option in the next step, whether the price goes up or down. The value of the option today *must* equal the value of this replicating portfolio.

This logic leads to the risk-neutral pricing formula. The value of the option at any node is the expected value of its future payoffs in the next step, discounted at the risk-free rate, using the risk-neutral probabilities:
$$ C_t = e^{-r \Delta t} [p C_u + (1-p) C_d] $$ 
where $C_u$ and $C_d$ are the option values in the up and down states, respectively. We can solve for the option price by starting at the final nodes (where the value is the known payoff) and working backward through the tree to the present (time 0). The diagram below visualizes this backward induction process for a small 3-step tree.

![Binomial Tree Visualization](images/finance/options/binomial_tree_viz.png)
*Figure 2: A 3-step binomial tree showing the stock price (S) and option price (C) at each node, calculated via backward induction.*

<a id='bsm'></a>
## 4. Model 2: The Black-Scholes-Merton (BSM) Model

The Black-Scholes-Merton model is the continuous-time limit of the binomial model as the number of time steps approaches infinity. It assumes the underlying asset price follows a **Geometric Brownian Motion (GBM)**:
$$ dS_t = \mu S_t dt + \sigma S_t dW_t $$
where $\mu$ is the drift rate, $\sigma$ is the volatility, and $dW_t$ is a Wiener process.

<a id='bsm-pde'></a>
### Deriving the BSM Partial Differential Equation

The derivation is a cornerstone of financial engineering. We construct a portfolio, $\Pi$, consisting of one option and a short position of $\Delta$ shares of the underlying stock:
$$ \Pi = C(S,t) - \Delta S $$
The change in the portfolio's value, $d\Pi$, over a small time step $dt$ comes from the change in the option's value and the change in the stock's value. Using **Itô's Lemma** for the option price $C(S,t)$, we get:
$$ dC = \left( \frac{\partial C}{\partial t} + \mu S \frac{\partial C}{\partial S} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 C}{\partial S^2} \right) dt + \sigma S \frac{\partial C}{\partial S} dW_t $$
The change in the portfolio is then:
$$ d\Pi = dC - \Delta dS = \left( \frac{\partial C}{\partial t} + \mu S \frac{\partial C}{\partial S} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 C}{\partial S^2} - \Delta \mu S \right) dt + (\sigma S \frac{\partial C}{\partial S} - \Delta \sigma S) dW_t $$
Now, we choose $\Delta = \frac{\partial C}{\partial S}$ (this is the option's delta). This choice cleverly eliminates the random term ($dW_t$), making the portfolio's return risk-free over the small interval $dt$.
$$ d\Pi = \left( \frac{\partial C}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 C}{\partial S^2} \right) dt $$
Since the portfolio is now risk-free, to prevent arbitrage, it must earn the risk-free rate, $r$. Its return must equal $r\Pi dt$:
$$ d\Pi = r \Pi dt = r \left( C - \frac{\partial C}{\partial S} S \right) dt $$
Equating the two expressions for $d\Pi$ and cancelling $dt$ yields the celebrated **Black-Scholes-Merton Partial Differential Equation (PDE)**:
$$ \frac{\partial C}{\partial t} + rS \frac{\partial C}{\partial S} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 C}{\partial S^2} - rC = 0 $$
This PDE governs the price of any derivative on a non-dividend-paying stock. The specific derivative (e.g., call vs. put) is defined by the **boundary conditions** of the problem (i.e., the known payoff at expiration).

<a id='bsm-formula'></a>
### The BSM Formula for European Options

Solving the BSM PDE with the boundary condition for a European call option ($C(S,T) = \max(S_T - K, 0)$) gives the famous BSM formula:
$$ C(S, t) = S_t N(d_1) - K e^{-r(T-t)} N(d_2) $$
where:
- $N(\cdot)$ is the cumulative distribution function (CDF) of the standard normal distribution.
- $d_1 = \frac{\ln(S_t/K) + (r + \frac{1}{2}\sigma^2)(T-t)}{\sigma\sqrt{T-t}}$
- $d_2 = d_1 - \sigma\sqrt{T-t}$

The formula has a beautiful financial interpretation: $S_t N(d_1)$ is the present value of receiving the stock if the option finishes in-the-money, and $K e^{-r(T-t)} N(d_2)$ is the present value of paying the strike price. $N(d_1)$ and $N(d_2)$ are risk-adjusted probabilities. In particular, $N(d_1)$ acts as the option's Delta.

### Code Implementation: A Class-Based Approach

To organize our work, we will use a class-based structure. A base class `OptionPricer` will hold the common parameters, and specific models will inherit from it.

In [ ]:
sec("Option Pricer Classes")

class OptionPricer:
    """Base class for option pricing models."""
    def __init__(self, S, K, T, r, sigma, option_type='call'):
        self.S, self.K, self.T, self.r, self.sigma = S, K, T, r, sigma
        if option_type not in ['call', 'put']:
            raise ValueError("Option type must be 'call' or 'put'.")
        self.option_type = option_type

class BinomialPricer(OptionPricer):
    """Prices an option using a Cox-Ross-Rubinstein binomial tree model."""
    def price(self, n_steps, exercise_type='european'):
        dt = self.T / n_steps
        u = np.exp(self.sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp(self.r * dt) - d) / (u - d)
        
        asset_prices = self.S * (u**np.arange(n_steps, -1, -1)) * (d**np.arange(0, n_steps + 1, 1))
        
        option_values = np.maximum(asset_prices - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - asset_prices, 0)
        
        for i in range(n_steps - 1, -1, -1):
            option_values = (p * option_values[:-1] + (1 - p) * option_values[1:]) * np.exp(-self.r * dt)
            if exercise_type == 'american':
                current_prices = self.S * (u**np.arange(i, -1, -1)) * (d**np.arange(0, i + 1, 1))
                intrinsic = np.maximum(current_prices - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - current_prices, 0)
                option_values = np.maximum(option_values, intrinsic)
        return option_values[0]

class BSMPricer(OptionPricer):
    """Prices a European option using the Black-Scholes-Merton formula."""
    def _get_d1_d2(self):
        if self.T <= 1e-9: return np.inf, np.inf
        d1 = (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)
        return d1, d2

    def price(self):
        if self.T <= 1e-9: return np.maximum(0.0, self.S - self.K) if self.option_type == 'call' else np.maximum(0.0, self.K - self.S)
        d1, d2 = self._get_d1_d2()
        if self.option_type == 'call':
            return self.S * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        else:
            return self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S * norm.cdf(-d1)

    def get_greeks(self):
        if self.T <= 1e-9: return {k: 0 for k in ['delta', 'gamma', 'vega', 'theta', 'rho']}
        d1, d2 = self._get_d1_d2()
        N_prime_d1 = norm.pdf(d1)
        greeks = {
            'gamma': N_prime_d1 / (self.S * self.sigma * np.sqrt(self.T)),
            'vega': self.S * np.sqrt(self.T) * N_prime_d1 / 100 # per 1% change
        }
        if self.option_type == 'call':
            greeks['delta'] = norm.cdf(d1)
            greeks['theta'] = (-(self.S * N_prime_d1 * self.sigma) / (2 * np.sqrt(self.T)) - self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(d2)) / 365
            greeks['rho'] = self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2) / 100
        else: # Put
            greeks['delta'] = norm.cdf(d1) - 1
            greeks['theta'] = (-(self.S * N_prime_d1 * self.sigma) / (2 * np.sqrt(self.T)) + self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-d2)) / 365
            greeks['rho'] = -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2) / 100
        return greeks

class MonteCarloPricer(OptionPricer):
    """Prices an option using Monte Carlo simulation."""
    def _generate_paths(self, n_sims, n_steps, seed):
        rng = np.random.default_rng(seed)
        dt = self.T / n_steps
        Z = rng.standard_normal((n_sims, n_steps))
        paths = np.zeros((n_sims, n_steps + 1)); paths[:, 0] = self.S
        for t in range(1, n_steps + 1):
            paths[:, t] = paths[:, t-1] * np.exp((self.r - 0.5 * self.sigma**2) * dt + self.sigma * np.sqrt(dt) * Z[:, t-1])
        return paths

    def price(self, n_sims=100000, n_steps=1, seed=42):
        paths = self._generate_paths(n_sims, n_steps, seed)
        ST = paths[:, -1]
        payoffs = np.maximum(ST - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - ST, 0)
        return np.exp(-self.r * self.T) * np.mean(payoffs)
    
note("Option pricing classes defined.")

<a id='greeks'></a>
## 5. Risk Management: The "Greeks"

The "Greeks" are the partial derivatives of the option price with respect to its parameters. They are essential for quantifying and hedging the various risks in an options portfolio.

- **Delta ($\Delta = \frac{\partial C}{\partial S}$):** Measures the rate of change of the option price with respect to a change in the underlying stock price. A delta of 0.6 means the option price will increase by about $0.60 for a $1 increase in the stock price. It is the primary measure of directional exposure.
- **Gamma ($\Gamma = \frac{\partial^2 C}{\partial S^2}$):** Measures the rate of change of Delta. High Gamma indicates the hedge is very sensitive to market movements and needs to be rebalanced frequently.
- **Vega ($\nu = \frac{\partial C}{\partial \sigma}$):** Measures sensitivity to volatility. It is typically quoted as the change in option price for a 1 percentage point change in implied volatility.
- **Theta ($\Theta = -\frac{\partial C}{\partial t}$):** Measures the rate of price decay with the passage of time (time decay). For a long option position, theta is almost always negative.
- **Rho ($\rho = \frac{\partial C}{\partial r}$):** Measures sensitivity to the risk-free interest rate.

<a id='greeks-viz'></a>
### Visualizing the Greeks

Let's visualize how the primary Greeks for a call option change as the underlying stock price moves.

In [ ]:
sec("Visualizing the Greeks")
S_range = np.linspace(50, 150, 100)
K_strike, T_exp, r_rate, vol = 100.0, 1.0, 0.05, 0.2
greeks_data = {'delta': [], 'gamma': [], 'vega': [], 'theta': []}

for s_val in S_range:
    pricer = BSMPricer(s_val, K_strike, T_exp, r_rate, vol, 'call')
    greeks = pricer.get_greeks()
    for key in greeks_data: greeks_data[key].append(greeks[key])

fig, axs = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Figure 2: Option Greeks vs. Underlying Price for a European Call", fontsize=18, y=1.0)

axs[0, 0].plot(S_range, greeks_data['delta']); axs[0, 0].set_title('a) Delta', fontsize=14)
axs[0, 1].plot(S_range, greeks_data['gamma']); axs[0, 1].set_title('b) Gamma', fontsize=14)
axs[1, 0].plot(S_range, greeks_data['vega']); axs[1, 0].set_title('c) Vega', fontsize=14)
axs[1, 1].plot(S_range, greeks_data['theta']); axs[1, 1].set_title('d) Theta', fontsize=14)

for ax in axs.flat:
    ax.set_xlabel('Stock Price ($)')
    ax.axvline(K_strike, color='k', linestyle='--', lw=1.5, label='Strike Price')
    ax.legend()

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()

<a id='monte-carlo'></a>
## 6. Model 3: Monte Carlo Simulation for Option Pricing

When an option's payoff is path-dependent or too complex for a closed-form solution, Monte Carlo simulation becomes an essential tool. The method involves:
1.  Simulating a large number of possible price paths for the underlying asset under the risk-neutral measure.
2.  Calculating the option's payoff for each simulated path.
3.  Averaging the discounted payoffs to find the option's price.

<a id='exotics'></a>
### Pricing Exotic Options: Asian and Barrier Options
Let's extend our `MonteCarloPricer` to handle two common exotic options:
- **Asian Option:** The payoff depends on the average price of the underlying over the option's life. This makes it cheaper than a standard option and useful for hedging exposures that depend on an average price.
- **Barrier Option:** The option is either activated or extinguished if the underlying asset price crosses a predetermined barrier level. An 'up-and-out' put option, for example, becomes worthless if the price goes above the barrier.

In [ ]:
sec("Monte Carlo Pricing for Exotic Options")

def price_asian(self, n_sims=20000, n_steps=100, seed=42):
    paths = self._generate_paths(n_sims, n_steps, seed)
    avg_prices = np.mean(paths[:, 1:], axis=1)
    payoffs = np.maximum(avg_prices - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - avg_prices, 0)
    return np.exp(-self.r * self.T) * np.mean(payoffs)

def price_barrier(self, barrier_level, barrier_type='up-and-out', n_sims=20000, n_steps=100, seed=42):
    paths = self._generate_paths(n_sims, n_steps, seed)
    final_prices = paths[:, -1]
    
    # Check if barrier was hit
    if barrier_type == 'up-and-out':
        barrier_hit = np.any(paths > barrier_level, axis=1)
        payoffs = np.maximum(self.K - final_prices, 0) if self.option_type == 'put' else np.maximum(final_prices - self.K, 0)
        payoffs[barrier_hit] = 0 # Option is knocked out
    elif barrier_type == 'down-and-in':
        barrier_hit = np.any(paths < barrier_level, axis=1)
        payoffs = np.maximum(final_prices - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - final_prices, 0)
        payoffs[~barrier_hit] = 0 # Option is never knocked in
    else:
        raise ValueError("Unsupported barrier type")
        
    return np.exp(-self.r * self.T) * np.mean(payoffs)

# Add new methods to the class
MonteCarloPricer.price_asian = price_asian
MonteCarloPricer.price_barrier = price_barrier

# --- Example Usage ---
S0, K, T, r, sigma = 100.0, 100.0, 1.0, 0.05, 0.2
mc_pricer = MonteCarloPricer(S0, K, T, r, sigma, 'call')
bsm_pricer = BSMPricer(S0, K, T, r, sigma, 'call')

european_price = bsm_pricer.price()
asian_price = mc_pricer.price_asian()
barrier_price = mc_pricer.price_barrier(barrier_level=120, barrier_type='up-and-out')

note(f"Standard European Call Price: ${european_price:.3f}")
note(f"Asian Call (Average Price) Price: ${asian_price:.3f}")
note(f"Up-and-Out Barrier Call (Barrier at $120) Price: ${barrier_price:.3f}")
note("As expected, the path-dependent options are cheaper. The Asian option is cheaper because the volatility of an average price is lower than the volatility of the final price. The barrier option is cheaper because there is a chance it becomes worthless before expiration.")

<a id='vol-smile'></a>
## 7. Real-World Application: The Volatility Smile

In the BSM model, volatility is a constant input. In reality, we can observe market prices of options and reverse-engineer the BSM formula to find the **implied volatility** ($\sigma_{imp}$)—the volatility level that makes the BSM price equal the market price. This is the market's consensus forecast of future volatility.

If the BSM model were a perfect description of reality, implied volatility would be constant across all strike prices for a given expiration. In reality, it is not. A plot of implied volatility against strike price reveals a **"volatility smile"** or **"smirk."** This pattern shows that out-of-the-money puts (low strikes) and calls (high strikes) are priced with a higher implied volatility than at-the-money options. This reflects the market's perception of a higher probability of large price moves (i.e., fatter tails in the return distribution) than the log-normal distribution assumed by BSM.

<a id='case-study'></a>
### Case Study: Calculating Implied Volatility for AAPL Options

Let's move from mock data to reality. We will fetch live option chain data for Apple Inc. (AAPL) using the `yfinance` library, calculate the implied volatility for each option, and plot the real-world volatility smirk.

In [ ]:
<a id='bsm-formula'></a>
### The BSM Formula for European Options

Solving the BSM PDE with the boundary condition for a European call option ($C(S,T) = \max(S_T - K, 0)$) gives the famous BSM formula:
$$ C(S, t) = S_t N(d_1) - K e^{-r(T-t)} N(d_2) $$
where:
- $N(\cdot)$ is the cumulative distribution function (CDF) of the standard normal distribution.
- $d_1 = \frac{\ln(S_t/K) + (r + \frac{1}{2}\sigma^2)(T-t)}{\sigma\sqrt{T-t}}$
- $d_2 = d_1 - \sigma\sqrt{T-t}$

The formula has a beautiful financial interpretation: $S_t N(d_1)$ is the present value of receiving the stock if the option finishes in-the-money, and $K e^{-r(T-t)} N(d_2)$ is the present value of paying the strike price. $N(d_1)$ and $N(d_2)$ are risk-adjusted probabilities. In particular, $N(d_1)$ acts as the option's Delta.

As the number of steps in the Binomial model approaches infinity, its price converges to the BSM price. We can demonstrate this numerically.

In [ ]:
sec("Convergence of Binomial Model to Black-Scholes")
S0, K, T, r, sigma = 100, 100, 1, 0.05, 0.2
bsm_price = BSMPricer(S0, K, T, r, sigma, 'call').price()

steps = np.arange(10, 501, 10)
binomial_prices = [BinomialPricer(S0, K, T, r, sigma, 'call').price(n_steps=n) for n in steps]

plt.figure(figsize=(14, 7))
plt.plot(steps, binomial_prices, label='Binomial Model Price')
plt.axhline(bsm_price, color='r', linestyle='--', label=f'Black-Scholes Price (${bsm_price:.4f})')
plt.title('Binomial Model Price Convergence to Black-Scholes')
plt.xlabel('Number of Steps in Binomial Tree')
plt.ylabel('Call Option Price')
plt.legend()
plt.show()

note("The plot clearly shows that as the number of time steps in the binomial tree increases, the calculated option price converges smoothly to the analytical Black-Scholes price. This provides a powerful visual confirmation of the theoretical link between the discrete-time and continuous-time models.")

### Code Implementation: A Class-Based Approach

To organize our work, we will use a class-based structure. A base class `OptionPricer` will hold the common parameters, and specific models will inherit from it.

In [ ]:
sec("Option Pricer Classes")

class OptionPricer:
    """Base class for option pricing models."""
    def __init__(self, S, K, T, r, sigma, option_type='call'):
        self.S, self.K, self.T, self.r, self.sigma = S, K, T, r, sigma
        if option_type not in ['call', 'put']:
            raise ValueError("Option type must be 'call' or 'put'.")
        self.option_type = option_type

class BinomialPricer(OptionPricer):
    """Prices an option using a Cox-Ross-Rubinstein binomial tree model."""
    def price(self, n_steps, exercise_type='european'):
        dt = self.T / n_steps
        u = np.exp(self.sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp(self.r * dt) - d) / (u - d)
        
        # Initialize asset prices at maturity
        asset_prices = self.S * (u**np.arange(n_steps, -1, -1)) * (d**np.arange(0, n_steps + 1, 1))
        
        # Initialize option values at maturity
        option_values = np.maximum(asset_prices - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - asset_prices, 0)
        
        # Step backwards through the tree
        for i in range(n_steps - 1, -1, -1):
            option_values = (p * option_values[:-1] + (1 - p) * option_values[1:]) * np.exp(-self.r * dt)
            if exercise_type == 'american':
                current_prices = self.S * (u**np.arange(i, -1, -1)) * (d**np.arange(0, i + 1, 1))
                intrinsic_value = np.maximum(current_prices - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - current_prices, 0)
                option_values = np.maximum(option_values, intrinsic_value)
        return option_values[0]

class BSMPricer(OptionPricer):
    """Prices a European option using the Black-Scholes-Merton formula."""
    def _get_d1_d2(self):
        if self.T <= 1e-9: return np.inf, np.inf
        d1 = (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)
        return d1, d2

    def price(self):
        if self.T <= 1e-9: return np.maximum(0.0, self.S - self.K) if self.option_type == 'call' else np.maximum(0.0, self.K - self.S)
        d1, d2 = self._get_d1_d2()
        if self.option_type == 'call':
            return self.S * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        else:
            return self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S * norm.cdf(-d1)

    def get_greeks(self):
        if self.T <= 1e-9: return {k: 0 for k in ['delta', 'gamma', 'vega', 'theta', 'rho']}
        d1, d2 = self._get_d1_d2()
        N_prime_d1 = norm.pdf(d1)
        greeks = {
            'gamma': N_prime_d1 / (self.S * self.sigma * np.sqrt(self.T)),
            'vega': self.S * np.sqrt(self.T) * N_prime_d1 / 100 # per 1% change
        }
        if self.option_type == 'call':
            greeks['delta'] = norm.cdf(d1)
            greeks['theta'] = (-(self.S * N_prime_d1 * self.sigma) / (2 * np.sqrt(self.T)) - self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(d2)) / 365
            greeks['rho'] = self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2) / 100
        else: # Put
            greeks['delta'] = norm.cdf(d1) - 1
            greeks['theta'] = (-(self.S * N_prime_d1 * self.sigma) / (2 * np.sqrt(self.T)) + self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-d2)) / 365
            greeks['rho'] = -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2) / 100
        return greeks

class MonteCarloPricer(OptionPricer):
    """Prices an option using Monte Carlo simulation."""
    def _generate_paths(self, n_sims, n_steps, seed):
        rng = np.random.default_rng(seed)
        dt = self.T / n_steps
        Z = rng.standard_normal((n_sims, n_steps))
        paths = np.zeros((n_sims, n_steps + 1)); paths[:, 0] = self.S
        for t in range(1, n_steps + 1):
            paths[:, t] = paths[:, t-1] * np.exp((self.r - 0.5 * self.sigma**2) * dt + self.sigma * np.sqrt(dt) * Z[:, t-1])
        return paths

    def price(self, n_sims=100000, n_steps=1, seed=42):
        paths = self._generate_paths(n_sims, n_steps, seed)
        ST = paths[:, -1]
        payoffs = np.maximum(ST - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - ST, 0)
        return np.exp(-self.r * self.T) * np.mean(payoffs)
    
note("Option pricing classes defined.")

<a id='greeks'></a>
## 5. Risk Management: The "Greeks"

The "Greeks" are the partial derivatives of the option price with respect to its parameters. They are essential for quantifying and hedging the various risks in an options portfolio.

- **Delta ($\Delta = \frac{\partial C}{\partial S}$):** Measures the rate of change of the option price with respect to a change in the underlying stock price. A delta of 0.6 means the option price will increase by about $0.60 for a $1 increase in the stock price. It is the primary measure of directional exposure.
- **Gamma ($\Gamma = \frac{\partial^2 C}{\partial S^2}$):** Measures the rate of change of Delta. High Gamma indicates the hedge is very sensitive to market movements and needs to be rebalanced frequently.
- **Vega ($\nu = \frac{\partial C}{\partial \sigma}$):** Measures sensitivity to volatility. It is typically quoted as the change in option price for a 1 percentage point change in implied volatility.
- **Theta ($\Theta = -\frac{\partial C}{\partial t}$):** Measures the rate of price decay with the passage of time (time decay). For a long option position, theta is almost always negative.
- **Rho ($\rho = \frac{\partial C}{\partial r}$):** Measures sensitivity to the risk-free interest rate.

<a id='greeks-viz'></a>
### Visualizing the Greeks

Let's visualize how the primary Greeks for a call option change as the underlying stock price moves.

In [ ]:
sec("Visualizing the Greeks")
S_range = np.linspace(50, 150, 100)
K_strike, T_exp, r_rate, vol = 100.0, 1.0, 0.05, 0.2
greeks_data = {'delta': [], 'gamma': [], 'vega': [], 'theta': []}

for s_val in S_range:
    pricer = BSMPricer(s_val, K_strike, T_exp, r_rate, vol, 'call')
    greeks = pricer.get_greeks()
    for key in greeks_data: greeks_data[key].append(greeks[key])

fig, axs = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Figure 3: Option Greeks vs. Underlying Price for a European Call", fontsize=18, y=1.0)

axs[0, 0].plot(S_range, greeks_data['delta']); axs[0, 0].set_title('a) Delta', fontsize=14)
axs[0, 1].plot(S_range, greeks_data['gamma']); axs[0, 1].set_title('b) Gamma', fontsize=14)
axs[1, 0].plot(S_range, greeks_data['vega']); axs[1, 0].set_title('c) Vega', fontsize=14)
axs[1, 1].plot(S_range, greeks_data['theta']); axs[1, 1].set_title('d) Theta', fontsize=14)

for ax in axs.flat:
    ax.set_xlabel('Stock Price ($)')
    ax.axvline(K_strike, color='k', linestyle='--', lw=1.5, label='Strike Price')
    ax.legend()

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()

<a id='monte-carlo'></a>
## 6. Model 3: Monte Carlo Simulation for Option Pricing

When an option's payoff is path-dependent or too complex for a closed-form solution, Monte Carlo simulation becomes an essential tool. The method involves:
1.  Simulating a large number of possible price paths for the underlying asset under the risk-neutral measure.
2.  Calculating the option's payoff for each simulated path.
3.  Averaging the discounted payoffs to find the option's price.

<a id='exotics'></a>
### Pricing Exotic Options: Asian and Barrier Options
Let's extend our `MonteCarloPricer` to handle two common exotic options:
- **Asian Option:** The payoff depends on the average price of the underlying over the option's life. This makes it cheaper than a standard option and useful for hedging exposures that depend on an average price.
- **Barrier Option:** The option is either activated or extinguished if the underlying asset price crosses a predetermined barrier level. An 'up-and-out' put option, for example, becomes worthless if the price goes above the barrier.

In [ ]:
sec("Monte Carlo Pricing for Exotic Options")

def price_asian(self, n_sims=20000, n_steps=100, seed=42):
    paths = self._generate_paths(n_sims, n_steps, seed)
    avg_prices = np.mean(paths[:, 1:], axis=1)
    payoffs = np.maximum(avg_prices - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - avg_prices, 0)
    return np.exp(-self.r * self.T) * np.mean(payoffs)

def price_barrier(self, barrier_level, barrier_type='up-and-out', n_sims=20000, n_steps=100, seed=42):
    paths = self._generate_paths(n_sims, n_steps, seed)
    final_prices = paths[:, -1]
    
    # Check if barrier was hit
    if barrier_type == 'up-and-out':
        barrier_hit = np.any(paths > barrier_level, axis=1)
        payoffs = np.maximum(self.K - final_prices, 0) if self.option_type == 'put' else np.maximum(final_prices - self.K, 0)
        payoffs[barrier_hit] = 0 # Option is knocked out
    elif barrier_type == 'down-and-in':
        barrier_hit = np.any(paths < barrier_level, axis=1)
        payoffs = np.maximum(final_prices - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - final_prices, 0)
        payoffs[~barrier_hit] = 0 # Option is never knocked in
    else:
        raise ValueError("Unsupported barrier type")
        
    return np.exp(-self.r * self.T) * np.mean(payoffs)

# Add new methods to the class
MonteCarloPricer.price_asian = price_asian
MonteCarloPricer.price_barrier = price_barrier

# --- Example Usage ---
S0, K, T, r, sigma = 100.0, 100.0, 1.0, 0.05, 0.2
mc_pricer = MonteCarloPricer(S0, K, T, r, sigma, 'call')
bsm_pricer = BSMPricer(S0, K, T, r, sigma, 'call')

european_price = bsm_pricer.price()
asian_price = mc_pricer.price_asian()
barrier_price = mc_pricer.price_barrier(barrier_level=120, barrier_type='up-and-out')

note(f"Standard European Call Price: ${european_price:.3f}")
note(f"Asian Call (Average Price) Price: ${asian_price:.3f}")
note(f"Up-and-Out Barrier Call (Barrier at $120) Price: ${barrier_price:.3f}")
note("As expected, the path-dependent options are cheaper. The Asian option is cheaper because the volatility of an average price is lower than the volatility of the final price. The barrier option is cheaper because there is a chance it becomes worthless before expiration.")

<a id='vol-smile'></a>
## 7. Real-World Application: The Volatility Smile

In the BSM model, volatility is a constant input. In reality, we can observe market prices of options and reverse-engineer the BSM formula to find the **implied volatility** ($\sigma_{imp}$)—the volatility level that makes the BSM price equal the market price. This is the market's consensus forecast of future volatility.

If the BSM model were a perfect description of reality, implied volatility would be constant across all strike prices for a given expiration. In reality, it is not. A plot of implied volatility against strike price reveals a **"volatility smile"** or **"smirk."** This pattern shows that out-of-the-money puts (low strikes) and calls (high strikes) are priced with a higher implied volatility than at-the-money options. This reflects the market's perception of a higher probability of large price moves (i.e., fatter tails in the return distribution) than the log-normal distribution assumed by BSM.

<a id='case-study'></a>
### Case Study: Calculating Implied Volatility for AAPL Options

Let's move from mock data to reality. We will fetch live option chain data for Apple Inc. (AAPL) using the `yfinance` library, calculate the implied volatility for each option, and plot the real-world volatility smirk.

In [ ]:
sec("Implied Volatility and the Volatility Smile")

def implied_volatility(market_price, S, K, T, r, option_type):
    # Objective function: difference between BSM price and market price
    objective = lambda sigma: BSMPricer(S, K, T, r, sigma, option_type).price() - market_price
    try:
        # Use a root-finding algorithm to find the sigma that makes the objective zero
        return brentq(objective, 1e-6, 5.0) # Search for vol between 0.01% and 500%
    except (ValueError, RuntimeError):
        return np.nan

def plot_volatility_smile(ticker_symbol="AAPL"):
    if not YFINANCE_AVAILABLE:
        note(f"Skipping volatility smile plot for {ticker_symbol} because `yfinance` is not installed.")
        return
    
    try:
        # 1. Fetch data for the stock
        ticker = yf.Ticker(ticker_symbol)
        S_market = ticker.history(period='1d')['Close'].iloc[-1]
        exp_dates = ticker.options
        # Select a near-term expiration date (e.g., the 3rd available one)
        options = ticker.option_chain(exp_dates[2])
        calls = options.calls
        puts = options.puts
        exp_date_str = exp_dates[2]
        
        # 2. Calculate time to maturity and risk-free rate
        T_market = (pd.to_datetime(exp_date_str) - pd.to_datetime('today')).days / 365.0
        r_market = yf.Ticker('^IRX').history(period='1d')['Close'].iloc[-1] / 100
        
    except Exception as e:
        note(f"Could not fetch live option data for {ticker_symbol} from yfinance (Error: {e}). Using illustrative data instead.")
        # Fallback to illustrative data
        S_market, T_market, r_market = 170.0, 0.1, 0.05
        strikes = np.arange(140, 201, 5)
        vols = 0.35 - 0.15 * ((strikes - S_market) / S_market) + 0.5 * ((strikes - S_market) / S_market)**2
        calls = pd.DataFrame({'strike': strikes, 'impliedVolatility': vols})
        puts = pd.DataFrame({'strike': strikes, 'impliedVolatility': vols + 0.02})
        exp_date_str = "Illustrative Data"

    # 3. Calculate implied volatility for calls and puts
    for df, opt_type in zip([calls, puts], ['call', 'put']):
        df['mid_price'] = (df['bid'] + df['ask']) / 2 if 'bid' in df.columns else np.nan
        df.dropna(subset=['mid_price', 'strike'], inplace=True)
        df = df[df['mid_price'] > 0]
        df['iv'] = df.apply(lambda row: implied_volatility(row['mid_price'], S_market, row['strike'], T_market, r_market, opt_type), axis=1)

    # 4. Plot the volatility smile
    plt.figure(figsize=(14, 8))
    plt.plot(calls['strike'], calls['iv'] * 100, marker='o', linestyle='--', label='Calls')
    plt.plot(puts['strike'], puts['iv'] * 100, marker='x', linestyle=':', label='Puts')
    plt.title(f"Figure 4: Implied Volatility Smirk for {ticker_symbol} Options (Expiration: {exp_date_str})", fontsize=18)
    plt.xlabel('Strike Price (K)'); plt.ylabel('Implied Volatility (%)')
    plt.axvline(S_market, color='k', linestyle='--', lw=1.5, label=f'Current Stock Price (${S_market:.2f})')
    plt.legend(); plt.grid(True)
    plt.show()

    note("The plot shows a classic volatility 'smirk' for equity options. Implied volatility is highest for low-strike (out-of-the-money) puts and decreases as the strike price increases. This reflects higher market demand for crash protection (puts), implying that the market prices in a higher probability of large downward moves than the log-normal distribution of BSM assumes.")

plot_volatility_smile()

<a id='beyond-bsm'></a>
## 8. Beyond Black-Scholes: Handling the Smile

The existence of the volatility smile is a clear signal that the assumptions of the BSM model (particularly constant volatility and log-normal returns) are violated in reality. This has led to the development of more advanced models that can account for these features:

1.  **Stochastic Volatility Models (e.g., Heston Model):** These models assume that volatility itself is a random variable that follows its own stochastic process. When volatility is high, large price swings are more likely, and when it's low, prices are more stable. This can generate the "smirk" seen in equity options, as it allows for a correlation between price shocks and volatility shocks (leverage effect).

2.  **Jump-Diffusion Models (e.g., Merton's Model):** These models add a "jump" component to the asset price process, allowing for sudden, large, discontinuous movements (like a market crash). This explicitly adds "fat tails" to the return distribution, making extreme events more probable than in the BSM world and thus generating a volatility smile.

3.  **Local Volatility Models:** These models assume that volatility is a deterministic function of both the current asset price and time, $\sigma(S, t)$. The function is calibrated to match the observed implied volatility surface perfectly. While they fit the market perfectly by construction, their predictive power can be limited.

These advanced models are the workhorses of modern quantitative finance, providing a more realistic framework for pricing and hedging complex derivatives.

<a id='exercises'></a>
## 9. Exercises

1.  **BSM PDE Verification:** Show that the BSM formula for a call option is indeed a solution to the BSM PDE. This requires calculating the partial derivatives of the BSM formula ($\Theta, \Delta, \Gamma$) and substituting them into the PDE to confirm that the equation equals zero.

2.  **American vs. European Puts:** Using the `BinomialPricer`, calculate and compare the price of a European put and an American put with `S=100, K=110, r=0.05, sigma=0.25, T=1`. Why is the American put worth more? Is an American *call* on a non-dividend-paying stock ever worth more than its European counterpart? Why or why not?

3.  **Put-Call Parity:** The put-call parity relationship for European options is $C - P = S - Ke^{-rT}$. Using the `BSMPricer`, create a call pricer and a put pricer with the same parameters. Verify that the prices they produce satisfy this no-arbitrage condition. What happens to this relationship for American options?

4.  **Exotic Pricing Challenge:** Implement the pricing of a 'down-and-in' call option in the `MonteCarloPricer` class. This option only comes into existence if the stock price drops below a certain barrier. Price it with `S=100, K=100, T=1, r=0.05, sigma=0.2, barrier=90`. How does its price compare to a standard European call?

5. **Interpreting the Smirk:** In the AAPL case study, we observed a volatility smirk. If you were a trader, how would you use this information? If you believed that the market was overestimating the probability of a crash, what options strategy could you construct to profit from this view? (Hint: Think about selling overpriced insurance).